# Single Inputs and Outputs - Pad, Color

In [1]:
import numpy as np
import pandas as pd
import torch
import os
import json
from pathlib import Path
import re
import itertools
from itertools import product
from torchtext import data
import torch.nn.functional as F
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
from matplotlib import colors

# thank you to "visualizing all task pairs with gridlines" and "starter notebook"!


In [2]:
# for dirname, _, filenames in os.walk('/storage/data'):
#     print(dirname)

In [3]:
# Define Paths
# 
# data_path = Path('/kaggle/input/abstraction-and-reasoning-challenge/')
# here it's in /storage/data

data_path = Path('/storage/data/')
training_path = data_path / 'training'
evaluation_path = data_path / 'evaluation'
test_path = data_path / 'test'
augment_path = data_path / 'augment'
fulltask_path = augment_path / 'fulltask'
subtask_path = augment_path / 'subtask'

substring = '.json'
baseName = ""

# test - make a directory 'foo'

# define the list of tasks we're going to be iterating through
training_tasks = sorted(os.listdir(training_path))


# training_path is /storage/data/training
# print(training_path) 
# print(type(training_path))


In [4]:
def showSizeIncrease(originalInputOrOutputSize, paddedInputOrOutputSize):
    # will return the '0r1c' part of  tra_0_25d8a9c8_D1_0r1c.png or .json

    #  trainingSizeIncrease = showSizeIncrease(originalTrainingInputSize, paddedTrainingInputSize)
    originalRows = originalInputOrOutputSize[0]
    originalColumns = originalInputOrOutputSize[1]
    paddedRows = paddedInputOrOutputSize[0]
    paddedColumns = paddedInputOrOutputSize[1]
    rowDifference = (paddedRows - originalRows)
    # cast to int so we don't have decimal points
    rowDifference = int(rowDifference)
    columnDifference = int(paddedColumns - originalColumns)
    columnDifference = int(columnDifference)
    strRowDifference = str(rowDifference)
    strColumnDifference = str(columnDifference)
    sizeIncreaseText = "-" + strRowDifference + "r" + strColumnDifference + "c"
    return sizeIncreaseText

Plot the subtask, which is an input or output pair

In [5]:
def plot_single(task, filename="harry"):
    cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
    norm = colors.Normalize(vmin=0, vmax=9)
    # 0 black, 1 blue, 2 red, 3 green, 4 yellow, 5 grey, 6 pink, 7 orange, 8 light blue, 9 brown
    # variable assignment: assigns 1 to fig and 2 to ax
    fig, ax = plt.subplots(1, 1)
    ax.imshow(task, cmap=cmap, norm=norm)
    width = np.shape(task)[1]
    height = np.shape(task)[0]
    ax.set_xticks(np.arange(-.5,width))
    ax.set_yticks(np.arange(-.5,height))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
#         fig.axes.get_xaxis().set_visible(False)
#         fig.axes.get_yaxis().set_visible(False)
    ax.tick_params(length=0)
    ax.grid(True)
    plt.tight_layout()
#     file_name_with_path = os.path.join(str(augment_path), filename)

    # name the directory with the tasks's baseName TODO this duplicate code from write single file 
    # should be in create_subtasks function but I couldn't get it to work
    
    directory = baseName
    # Parent Directory path
    parent_dir = str(augment_path)
    # Path
    path = Path(os.path.join(parent_dir, directory))
    taskDirectory = path.mkdir(parents=True, exist_ok=True)
    file_name_with_path = os.path.join(str(path), filename)

    plt.savefig(file_name_with_path + '.png')
    #include this next line if  you want the image to show in this notebook
#     plt.show()
    plt.close()
    

***Set Padding***

In [6]:
PAD = 4
arr = list(range(0,PAD + 1))
padded_train_pairs = []
padded_test_pairs = []
dict_all_padded = {}
adder = 0
# print(arr)
color_arr = list(range(0,10))
print(color_arr)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [7]:
def cartesian_product(arr): 

    # return the list of all the computed tuples 
    # using the product() method 
    return list(product(arr, arr, arr, arr)) 

cartesianList = cartesian_product(arr)
print(len(cartesianList))
print(cartesianList)

625
[(0, 0, 0, 0), (0, 0, 0, 1), (0, 0, 0, 2), (0, 0, 0, 3), (0, 0, 0, 4), (0, 0, 1, 0), (0, 0, 1, 1), (0, 0, 1, 2), (0, 0, 1, 3), (0, 0, 1, 4), (0, 0, 2, 0), (0, 0, 2, 1), (0, 0, 2, 2), (0, 0, 2, 3), (0, 0, 2, 4), (0, 0, 3, 0), (0, 0, 3, 1), (0, 0, 3, 2), (0, 0, 3, 3), (0, 0, 3, 4), (0, 0, 4, 0), (0, 0, 4, 1), (0, 0, 4, 2), (0, 0, 4, 3), (0, 0, 4, 4), (0, 1, 0, 0), (0, 1, 0, 1), (0, 1, 0, 2), (0, 1, 0, 3), (0, 1, 0, 4), (0, 1, 1, 0), (0, 1, 1, 1), (0, 1, 1, 2), (0, 1, 1, 3), (0, 1, 1, 4), (0, 1, 2, 0), (0, 1, 2, 1), (0, 1, 2, 2), (0, 1, 2, 3), (0, 1, 2, 4), (0, 1, 3, 0), (0, 1, 3, 1), (0, 1, 3, 2), (0, 1, 3, 3), (0, 1, 3, 4), (0, 1, 4, 0), (0, 1, 4, 1), (0, 1, 4, 2), (0, 1, 4, 3), (0, 1, 4, 4), (0, 2, 0, 0), (0, 2, 0, 1), (0, 2, 0, 2), (0, 2, 0, 3), (0, 2, 0, 4), (0, 2, 1, 0), (0, 2, 1, 1), (0, 2, 1, 2), (0, 2, 1, 3), (0, 2, 1, 4), (0, 2, 2, 0), (0, 2, 2, 1), (0, 2, 2, 2), (0, 2, 2, 3), (0, 2, 2, 4), (0, 2, 3, 0), (0, 2, 3, 1), (0, 2, 3, 2), (0, 2, 3, 3), (0, 2, 3, 4), (0, 2, 4, 0), (

**We want to rewrite the original subtask into a padded subttask, name it, and then write out the JSON and the image following naming conventions from Subtask notebook.
We will also write out a file for each training pair and each test pair**

In [8]:
PAD = 1
arr = list(range(-PAD,PAD + 1))
padded_train_pairs = []
padded_test_pairs = []
dict_all_padded = {}


In [9]:
def create_padding(task, n_train_pairs, n_test_pairs, baseName="newPaddingName"):

#     print("base name in create_padding: " + baseName, " n_train_pairs: ", n_train_pairs)
    
#     for i in range(len(cartesianList)):
    for i in range(0,len(cartesianList)):
    # makePaddingString(yShift, xShift) ... return yChangeString, xChangeString
    # so yChangeString will be renamedPaddingInput[0] and xChangeString is renamedPaddingInput[1]

        leftShift = cartesianList[i][0]
        rightShift = cartesianList[i][1]
        topsShift = cartesianList[i][2]
        bottomShift = cartesianList[i][3]

        # loop through training pairs
        training_pair_counter = 0
        # dictionary object to hold our modified padded task. Iterate through each training pair and
        # 
        padded_train_pairs = []
        for training_pair_counter in range(n_train_pairs):            
            # Input
            originalTrainingInput = task['train'][training_pair_counter]['input']
            # convert it to a tensor...
            inputTrainingTensor = torch.tensor(originalTrainingInput)
            # ... and get its size ...
            originalTrainingInputSize = list(inputTrainingTensor.size())
#             print("originalTrainingInputSize is ", originalTrainingInputSize)
            # ... so that we can pad it pad=(left, right, top, bottom)
            paddedTrainingInput = F.pad(input=inputTrainingTensor, pad=(leftShift,rightShift, topsShift, bottomShift), mode='constant', value=0)
            # create variable to hold this newly padded size
            paddedTrainingInputSize = list(paddedTrainingInput.size())
#             print("paddedTrainingInputSize is ", paddedTrainingInputSize)
            inputTrainingSizeIncrease = showSizeIncrease(originalTrainingInputSize, paddedTrainingInputSize)
    
            # Output
            originalTrainingOutput = task['train'][training_pair_counter]['output']
            # convert it to a tensor...
            outputTrainingTensor = torch.tensor(task['train'][training_pair_counter]['output'])
            # ... and get its size ...
            originalTrainingOutputSize = list(outputTrainingTensor.size())
            # ... so that we can pad it
            paddedTrainingOutput = F.pad(input=outputTrainingTensor, pad=(leftShift,rightShift, topsShift, bottomShift), mode='constant', value=0)
            # create variable to hold this newly padded size
            paddedTrainingOutputSize = list(paddedTrainingOutput.size())
            outputTrainingSizeIncrease = showSizeIncrease(originalTrainingOutputSize, paddedTrainingOutputSize)
            padded_train_pairs.append([paddedTrainingInput, paddedTrainingOutput])
            # test
        padded_test_pairs = []
        for test_pair_counter in range(n_test_pairs):
            # Input
            originalTestInput = task['test'][test_pair_counter]['input']
            inputTestTensor = torch.tensor(originalTestInput)
            originalTestInputSize = list(inputTestTensor.size())
            paddedTestInput = F.pad(input=inputTestTensor, pad=(leftShift,rightShift, topsShift, bottomShift), mode='constant', value=0)
            paddedTestInputSize = list(paddedTestInput.size())
            inputTestSizeIncrease = showSizeIncrease(originalTestInputSize, paddedTestInputSize)
#             print("inputTestSizeIncrease is ", inputTestSizeIncrease)
            paddedInputTestToTensor = torch.tensor(paddedTestInput)
            # Output
            originalTestOutput = task['test'][test_pair_counter]['output']
            outputTestTensor = torch.tensor(task['test'][test_pair_counter]['output'])
            originalTestOutputSize = list(outputTestTensor.size())
            paddedTestOutput = F.pad(input=outputTestTensor, pad=(leftShift,rightShift, topsShift, bottomShift), mode='constant', value=0)
            paddedTestOutputSize = list(paddedTestOutput.size())
            outputTestSizeIncrease = showSizeIncrease(originalTestOutputSize, paddedTestOutputSize)
            list_paddedTestOutputSize = list(paddedTestOutput.size())
            paddedOutputTestToTensor = torch.tensor(paddedTestOutput)

            padded_test_pairs.append([paddedInputTestToTensor, paddedOutputTestToTensor])
#         print("padded_test_pairs = ", padded_test_pairs)
#         print("padded_test_pairs length = ", len(padded_test_pairs))
#       now reassemble these matrices into a single padding task

        padded_train_dict = {'train': []}
        for item in padded_train_pairs:
            padded_train_dict['train'].append({
                'input': item[0].tolist(),
                'output': item[1].tolist()
            })
        padded_test_dict = {'test': []}
        for item in padded_test_pairs:
            padded_test_dict['test'].append({
                'input': item[0].tolist(),
                'output': item[1].tolist()
            })
#         print("padded_test_dict is ", padded_test_dict)

        dict_all_padded.update(padded_train_dict)
        dict_all_padded.update(padded_test_dict)
        sizeIncrease = inputTestSizeIncrease
        # the following line is if we want to write out the JSON
#         plot_task(dict_all_padded, padded_plot_name)
        create_subtasks(dict_all_padded,baseName, sizeIncrease)

In [10]:
def create_subtasks(task, baseName="newPaddingName", sizeIncrease="whatSizeIncrease?"):
#     print("baseName in create_subtasks is ", baseName)
# currently setting changeColor to True, but what we want is to run it once with it set to true, then once set to 
# false. 
    changeColor = True
# getting length of list 
    trainLength = len(task['train'])
    trainInputAndOutputPairs = task['train']
    testLength = len(task['test'])
    testInputAndOutputPairs = task['test']
    
    if changeColor:
        colorChange = "_cc" + str(adder)
    else:
        colorChange = ""
    
    # Iterating the index 
    for i in range(trainLength): 
        inputTraSinglePlotName =  baseName + 'tr' + str(i) + "0" + sizeIncrease + colorChange
        outputTraSinglePlotName = baseName + 'tr' + str(i) + "1" + sizeIncrease  + colorChange
        # tra_2_25d8a9c8_D2_1r2c.png
        inputTraSingleFileName = inputTraSinglePlotName  + ".lst"
        outputTraSingleFileName  = outputTraSinglePlotName + ".lst"
#         print("inputTraSinglePlotName in create_subtasks is", inputTraSinglePlotName)
        if changeColor:
            trainInputAndOutputPairs[i]['input'] = colorMyTensor(trainInputAndOutputPairs[i]['input'])
            trainInputAndOutputPairs[i]['output'] = colorMyTensor(trainInputAndOutputPairs[i]['output'])
        write_single_file(trainInputAndOutputPairs[i]['input'], inputTraSingleFileName)
        write_single_file(trainInputAndOutputPairs[i]['output'], outputTraSingleFileName)
        plot_single(trainInputAndOutputPairs[i]['input'], inputTraSinglePlotName)
        plot_single(trainInputAndOutputPairs[i]['output'], outputTraSinglePlotName)

    for i in range(testLength): 
#         print(testInputAndOutputPairs[i]['input'],  baseName + '_tst_input_' + str(i)) 
#         print(testInputAndOutputPairs[i]['output'],  baseName + '_tst_output_' + str(i)) 
        
        inputTstSinglePlotName  = baseName + 'ts' + str(i) + "0"  + sizeIncrease
        outputTstSinglePlotName = baseName + 'ts' + str(i) + "1" + sizeIncrease
        # names for plotted test subtasks
        inputTstSingleFileName = inputTstSinglePlotName + ".lst"
        outputTstSingleFileName = outputTstSinglePlotName + ".lst"

        write_single_file(testInputAndOutputPairs[i]['input'], inputTstSingleFileName)
        write_single_file(testInputAndOutputPairs[i]['output'], outputTstSingleFileName)
        plot_single(testInputAndOutputPairs[i]['input'], inputTstSinglePlotName)
        plot_single(testInputAndOutputPairs[i]['output'], outputTstSinglePlotName)

In [11]:
listToColor = [[8,9,8],
        [2, 3, 2],
        [7, 4, 7]]
# print(listToColor, type(listToColor))
tensorToColor = torch.tensor(listToColor)
changed_color_tensor = torch.add(tensorToColor,adder)
# uniqueValues = torch.unique(torch.tensor(tensorToColor, dtype=torch.long))
# uniqueValuesToArray = uniqueValues.numpy()
# uniqueValuesToArray
# boogie = np.random.choice(uniqueValuesToArray,2)
# print(type(uniqueValues.numpy()))
# boogie
#     don't think we need this next line, but if the one below it fails, try it
#     tensorToChange = torch.tensor(changed_color_tensor, dtype=torch.uint8)

# tensorToChange = torch.tensor(changed_color_tensor, dtype=torch.uint8)
changed_color_tensor[changed_color_tensor==10] = 0 #this means you added 1 to 9 and it's the only thing over 9
# To vary the number added, and to account for it:
changed_color_tensor[changed_color_tensor==11] = 1     #this means you added 2 to 9 so both 8 and 9 are over 9
changed_color_tensor[changed_color_tensor==12] = 2   #this means you added 3 so 7, 8, 9, and 10 are over 9
changed_color_tensor[changed_color_tensor==13] = 3   #added 4 to 9, so 6, 7, 8, 9, and 10 are over 9

# formula is changed_color_tensor[changed_color_tensor==10] = 0
# TODO - experiment with just changing the background color. That is, just changing 0 to 1, to 2, to 3, etc. 
# That is a variation on a larger generalized change: change just one color at a time. But that would probably
# only work if the adjacent color was not the same as the new one. 
changedColorList = changed_color_tensor.tolist()
print(changedColorList)


[[8, 9, 8], [2, 3, 2], [7, 4, 7]]


In [12]:
# This function shoud be called last, after an augmented tensor has been created

def colorMyTensor(listToColor):

    # adder is the offset to change color. If adder is 1, then 0 becomes 1, one becomes 2, etc. 
#     print("inputList = ", listToColor)
#     print(type(listToColor)) #list
    tensorToColor = torch.tensor(listToColor) #convert to pytorch tensor
    changed_color_tensor = torch.add(tensorToColor,adder)

    # the below is for testing:
    #     tensorToColor = [[8,9,8],
    #             [2, 3, 2],
    #             [7, 4, 7]]
    #     tensorToColor = torch.tensor(tensorToColor)
    #     changed_color_tensor = torch.add(tensorToColor,adder)
    # uniqueValues = torch.unique(torch.tensor(tensorToColor, dtype=torch.long))
    # uniqueValuesToArray = uniqueValues.numpy()
    # uniqueValuesToArray
    # boogie = np.random.choice(uniqueValuesToArray,2)
    # print(type(uniqueValues.numpy()))
    # boogie
    # don't think we need this next line, but if the one below it fails, try it
    # tensorToChange = torch.tensor(changed_color_tensor, dtype=torch.uint8)

#     tensorToChange = torch.tensor(changed_color_tensor, dtype=torch.uint8)
    changed_color_tensor[changed_color_tensor==10] = 0 #this means you added 1 to 9 and it's the only thing over 9
    # To vary the number added, and to account for it:
    changed_color_tensor[changed_color_tensor==11] = 1     #this means you added 2 to 9 so both 8 and 9 are over 9
    changed_color_tensor[changed_color_tensor==12] = 2   #this means you added 3 to 9 so 8, 9, and 10 are over 9
    # TODO - experiment with just changing the background color. That is, just changing 0 to 1, to 2, to 3, etc. 
    # That is a variation on a larger generalized change: change just one color at a time. But that would probably
    # only work if the adjacent color was not the same as the new one. 
    changedColorList = changed_color_tensor.tolist()
#     print("changedColorList = ", changedColorList)
    return changedColorList


In [13]:
def write_single_file(listOflists, filename):

    class NumpyEncoder(json.JSONEncoder):
        def default(self, obj):
            if isinstance(obj, np.ndarray):
                return obj.tolist()
            return json.JSONEncoder.default(self, obj)

    json_dump = json.dumps(listOflists, cls=NumpyEncoder)


    # name the directory with the tasks's baseName
    directory = baseName
    # Parent Directory path
    parent_dir = str(augment_path)
    # Path
    path = Path(os.path.join(parent_dir, directory))
    taskDirectory = path.mkdir(parents=True, exist_ok=True)
#     file_name_with_path = os.path.join(str(augment_path), filename)
    file_name_with_path = os.path.join(str(path), filename)


#     print("file_name_with_path in write_single_file: ", file_name_with_path)
    outfile = open(file_name_with_path, "w")
    outfile.write(json_dump)
    outfile.close()

In [ ]:
# Set how many training tasks we're going to iterate thru

for i, json_path in enumerate(training_tasks[13:20], start=13):
    
    task_file = str(training_path / json_path)
#     print("task_file in enumerate, ", task_file)
    with open(task_file, 'r') as f:
        task = json.load(f)
# print a list of task_files, e.g. 000 /storage/data/training/007bbfb7.json
#    print(f"{i:03d}", task_file)

    n_train_pairs = len(task['train'])
    n_test_pairs = len(task['test'])
#     task_file = str(training_path / training_tasks[i]) # the JSON file we're looking at, including its path
#     task is the full JSON object including train and test input and output 
#     print("training_tasks[i] in enumerate is ", training_tasks[i])
    if training_tasks[i].index(substring):
        idx = training_tasks[i].index(substring)
        if idx > 0:
            baseName = training_tasks[i][:idx]
#             print("substring: ", substring)
#             print("baseName in enumerate(training_tasks: ", baseName)

    create_padding(task, n_train_pairs, n_test_pairs, baseName)
    


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Let's do some transformation on this one's two inputs and outputs. We'll start with a flip:

In [ ]:
#  Let's do some transformation on this one's two inputs and outputs:
# original = task['train'][2]['input']
# flipped = (np.fliplr(original))

In [ ]:
# torch.tensor(original).shape